# 데이터 읽어오기

In [1]:
import glob
import os
import re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list=glob.glob(txt_file_path)

raw_corpus=[]

for txt_file in txt_list:
    with open(txt_file,'r') as f:
        raw=f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:",len(raw_corpus))
print('example:\n',raw_corpus[:3])


데이터 크기: 187088
example:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 정제

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1.소문자로 바꾸고, 양쪽 공백을 지운다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2.특수문자 양쪽에 공백을 넣는다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3.여러개의 공백을 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4.^a-zA-Z?.!,¿가 아니면 공백으로 바꾼다.
    sentence = sentence.strip() # 5.다시 양쪽 공백을 지운다.
    sentence = '<start> ' + sentence + ' <end>' # 6.문장 시작과 끝에 <start> <end>를 추가한다.
    return sentence


In [3]:
corpus=[]

for sentence in raw_corpus:
    if len(sentence)==0: continue # 공백 건너뛰기
    if sentence[-1]==':': continue # : 건너뛰기  
    preprocessed_sentence=preprocess_sentence(sentence)   
    corpus.append(preprocessed_sentence)

In [4]:
#corpus를 tensor로 변환
def tokenize(corpus):
    
    tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,
    filters='',
    oov_token="<unk>")
    
    
    tokenizer.fit_on_texts(corpus)
    tensor=tokenizer.texts_to_sequences(corpus)
    #입력데이터의 시퀀스 길이를  padding으로 맞춰줌, 최대 토큰 길이는 15
    tensor=tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=15,padding='post')
    
        
    print(tensor,tokenizer)
    return tensor,tokenizer

tensor,tokenizer=tokenize(corpus)
    

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f796f1fcac0>


In [5]:
src_input = tensor[:,:-1]
tgt_input=tensor[:,1:]

In [6]:
BUFFER_SIZE=len(src_input)
BATCH_SIZE=256
steps_per_epoch=len(src_input)//BATCH_SIZE

VOCAB_SIZE=tokenizer.num_words+1

dataset=tf.data.Dataset.from_tensor_slices((src_input,tgt_input))
dataset=dataset.shuffle(BUFFER_SIZE)
dataset=dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 평가 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=32
                                                          
)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


# 모델 만들기

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self,vocab_size,embedding_size,hidden_size):
        super().__init__()
        
        self.embedding=tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.rnn1=tf.keras.layers.LSTM(hidden_size,return_sequences=True)
        self.rnn2=tf.keras.layers.LSTM(hidden_size,return_sequences=True)
        self.linear=tf.keras.layers.Dense(vocab_size)
        
    def call(self,x):
        out=self.embedding(x)
        out=self.rnn1(out)
        out=self.rnn2(out)
        out=self.linear(out)
        
        return out

embedding_size=256
hidden_size=1024
model=TextGenerator(VOCAB_SIZE,embedding_size,hidden_size)

# 학습시키기

In [9]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
model.compile(loss=loss,optimizer=optimizer)
model.fit(dataset,epochs=10)

Epoch 1/10
686/686 [==============================] - 116s 161ms/step - loss: 3.6095
Epoch 2/10
686/686 [==============================] - 112s 163ms/step - loss: 3.1316
Epoch 3/10
686/686 [==============================] - 112s 163ms/step - loss: 2.9348
Epoch 4/10
686/686 [==============================] - 112s 162ms/step - loss: 2.7853
Epoch 5/10
686/686 [==============================] - 112s 163ms/step - loss: 2.6529
Epoch 6/10
686/686 [==============================] - 113s 164ms/step - loss: 2.5344
Epoch 7/10
686/686 [==============================] - 112s 163ms/step - loss: 2.4237
Epoch 8/10
686/686 [==============================] - 112s 163ms/step - loss: 2.3215
Epoch 9/10
686/686 [==============================] - 112s 163ms/step - loss: 2.2250
Epoch 10/10
686/686 [==============================] - 113s 164ms/step - loss: 2.1330


# 평가하기

In [10]:
def generate_text(model,tokenizer,init_sentence='<start>',max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
       
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
        
    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> you raise", max_len=20)

'<start> you raise my heart away <end> '